In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [83]:
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from preprocessing.utils import (
    load_event_comments,
    save_event_comments,
)

from load.utils import load_comments, save_df_as_parquet


In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")


In [9]:
EVENT_KEY = "mass_shootings"

event_comments = load_event_comments(EVENT_KEY)


In [21]:
embeddings = model.encode(
    event_comments["body_cleaned"].tolist(),
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=False,
)


Batches: 100%|██████████| 4/4 [00:08<00:00,  2.13s/it]


In [26]:
event_comments["embedding"] = embeddings

event_comments["embedding"] = event_comments["embedding"].apply(np.array)


In [ ]:
save_event_comments(event_comments, f"{EVENT_KEY}_with_embeddings")


(50, 384)

In [91]:
# group by user and aggregate using np mean (axis=0)
user_embeddings = event_comments.groupby(by="author")["embedding"].apply(
    lambda x: pd.Series(
        {
            "mean": np.mean(x, axis=0),
            "max": np.maximum.reduce(x.tolist(), axis=0),
        }
    )
).to_frame()


In [92]:
user_embeddings

embedding
author                                                        
1GnWK  mean  [-0.054766607, 0.059622414, 0.044726454, -0.07...
       max   [-0.054766607, 0.059622414, 0.044726454, -0.07...
1ciAb  mean  [0.057719804, 0.041568875, 0.01093863, -0.0424...
       max   [0.057719804, 0.041568875, 0.01093863, -0.0424...
1lLml  mean  [0.01731521, 0.052210815, -0.024826469, -0.040...
...                                                        ...
ybUwZ  max   [-0.014305461, 0.061665315, -0.0064223837, -0....
ydWux  mean  [-0.024960294, 0.05637507, 0.041508026, -0.062...
       max   [-0.024960294, 0.05637507, 0.041508026, -0.062...
zwo0h  mean  [0.029973043, -0.08774368, 0.045531765, -0.062...
       max   [0.029973043, -0.08774368, 0.045531765, -0.062...

[194 rows x 1 columns]

In [93]:
save_df_as_parquet(
    data=user_embeddings,
    target_file=f"{EVENT_KEY}_user_embeddings",
)
